In [1]:
#文章内の言葉の形態素解析
!pip install janome

     |████████████████████████████████| 19.7MB 1.3MB/s 


In [58]:
from google.colab import auth
from oauth2client.client import GoogleCredentials
import gspread

auth.authenticate_user()
gc = gspread.authorize(GoogleCredentials.get_application_default())

In [59]:
# ファイル名を指定してシートを開く
sht = gc.open('COVID19_tweet_sheet')
worksheet = sht.get_worksheet(0)

# 読み込むセルの範囲の指定(読み込みたい範囲がわかっているのであれば適宜書き換える)
row_cnt = worksheet.row_count
col_cnt = worksheet.col_count

cells = worksheet.range(1, 1, row_cnt, col_cnt)

table_data = []
cols = []

for i, cell in enumerate(cells):
  cols.append(cell.value)
  if (i + 1) % col_cnt == 0:
    table_data.append(cols)
    cols = []

print(table_data)

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [60]:
#記号を抜き取り、名詞だけをピックアップする。
import re
from janome.tokenizer import Tokenizer

def is_symbol(str):
  symbol = "!#&()+*-=,./:<>~!?@[]{}_^|」「。、"
  for s in symbol:
    if s in str:
      return True
  return False

t = Tokenizer()

token_list = []
for i in range(row_cnt):
  text = table_data[i][6]
  for token in t.tokenize(text):
    if token.part_of_speech.split(',')[0] == "名詞":
      if token.part_of_speech.split(',')[1] == "サ変接続" or token.part_of_speech.split(',')[1] == "一般":
        if token.surface.encode('utf-8').isalnum() == False and is_symbol(token.surface) == False:
          token_list.append(token.surface)

In [61]:
#どの言葉が一番多く使われたか。
import collections

collect = collections.Counter(token_list)
num_list = []
for i in range(len(collect.most_common())):
  tuple2list = list(collect.most_common()[i])
  num_list.append(tuple2list)
print(num_list)

[['コロナ', 12223], ['ウイルス', 11271], ['新型', 8738], ['感染', 6827], ['対策', 1672], ['月', 1410], ['確認', 1336], ['拡大', 928], ['影響', 865], ['発表', 792], ['人', 647], ['防止', 622], ['検査', 611], ['マスク', 598], ['県内', 558], ['情報', 543], ['ニュース', 538], ['患者', 534], ['状況', 509], ['発生', 479], ['予防', 473], ['質問', 468], ['台風', 466], ['世界', 437], ['お願い', 423], ['～', 384], ['開催', 373], ['死亡', 369], ['ワクチン', 359], ['陽性', 349], ['支援', 335], ['消毒', 312], ['医療', 302], ['避難', 302], ['中止', 291], ['対応', 288], ['関連', 276], ['男性', 276], ['効果', 267], ['実施', 264], ['接触', 263], ['市内', 263], ['営業', 261], ['新規', 259], ['着用', 253], ['お知らせ', 248], ['終息', 248], ['募集', 239], ['症状', 237], ['死者', 234], ['女性', 229], ['リンク', 223], ['インフルエンザ', 215], ['生活', 207], ['時点', 206], ['流行', 199], ['協力', 199], ['タ', 197], ['グ', 196], ['経済', 193], ['全国', 189], ['サイト', 189], ['匿名', 188], ['更新', 185], ['利用', 182], ['気', 182], ['国内', 180], ['病院', 179], ['減少', 177], ['速報', 172], ['企業', 171], ['関係', 170], ['社会', 169], ['判明', 168], ['予定', 165], ['研

In [63]:
row_len = len(num_list)
col_len = len(num_list[0])

sheet_name = "COVID19_word_num_sheet"

sh = gc.create(sheet_name)
worksheet = gc.open(sheet_name).sheet1

worksheet.add_rows(row_len)

cell_list = worksheet.range(1, 1, row_len, col_len)
col_list = [flatten for inner in num_list for flatten in inner]

for cell, col in zip(cell_list, col_list):
  cell.value = col

worksheet.update_cells(cell_list)

{'spreadsheetId': '1J0OJNyad92mKnld4u2EQgLmuX6XE5Nu3ygKRM6zu2W0',
 'updatedCells': 20716,
 'updatedColumns': 2,
 'updatedRange': "'シート1'!A1:B10358",
 'updatedRows': 10358}